___

<h1> Enunciado do Trabalho Prático (parte I) </h1>

___

<h2> Machine Learning </h2>
<h3> M. Sc. in Electrical and Computer Engineering </h3>
<h4> Instituto Superior de Engenharia / Universidade do Algarve </h4>

[MEEC](https://ise.ualg.pt/en/curso/1477) / [ISE](https://ise.ualg.pt) / [UAlg](https://www.ualg.pt)

Pedro J. S. Cardoso (pcardoso@ualg.pt)

___

Notas:
1. Leia atentatamente os textos (em markdown) e os comentários (no código) e responda às questões colocadas.
2. O trabalho deve ser
    - realizado em grupos de 2 alunos.
    - entregue até às 23h59 do dia 01 de Novembro de 2023, via Moodle.
    - realizado em Python, usando Jupyter Notebooks.
3. Entregue:
    - em formato Jupyter Notebook (`.ipynb`), com o nome `TP1_<axxxxx>-<nome1>_<axxxxx>-<nome2>.ipynb`, em que `<axxxxx>-<nome1>` e `<axxxxx>-<nome2>` são os número de alunos e respetivos nomes.
    - com todos os outputs, gráficos visíveis, comentários, etc.
    - com o código a funcionar, i.e., deve ser possível executar todas as células do notebook sem erros, de forma sequencial.
3. No final haverá uma sessão de perguntas e respostas sobre o trabalho. Será atribuida classificação negativa trabalhos que:
    - sejam cópias uns dos outros (0 valores - sem possibilidade de melhoria)
    - não sejam realizados pelos alunos que os entregam (0 valores - sem possibilidade de melhoria)
    - não sejam entregues até à data limite (desconto de 0.5 valores por dia de atraso, limite 4 valores)
    - não sejam apresentados na sessão de perguntas e respostas (0 valores)

# Inicialização
Comecem por definir os vossos nomes nas variáveis `nome_1` e `nome_2` e a seguir executem a célula seguinte para definir a variável `nomes_dos_alunos` que será usada mais à frente. Se for só um aluno, deixe a variável `nome_2` vazia.

In [ ]:
# TODO: coloquem aqui os vossos nomes
nome_1 = "Nome do aluno 1"
nome_2 = "Nome do aluno 2"

In [ ]:
nomes_dos_alunos = f"{nome_1} & {nome_2}"
f"Olá {nomes_dos_alunos}!"

Corra a seguinte célula que irá carregar um conjunto de funções que serão usadas mais à frente. Se tiver curiosidade, pode abrir o ficheiro `utils.ipynb` e ver o código que está lá dentro.

In [ ]:
%run 'utils.ipynb'

O seguinte código serve para definir a semente do vosso trabalho.

In [ ]:
seed = string_para_int_entre_0_e_b(nomes_dos_alunos, 10**6)
f"A vossa seed é {seed}"

E agora usamos a seed calculada atrás para definirmos a semente do gerador de números aleatórios, para que os resultados sejam reproduzíveis.

In [ ]:
import random
import numpy as np

random.seed(seed)
np.random.seed(seed)

## Classificação binária

Comecemos por criar um exemplo de resultado de classificação binária, com duas classes, 0 e 1. Imagine que tem emails e quer classificá-los em _spam_ (1) ou não _spam_ (0). Ou, quer fazer um teste que indica se uma pessoa tem ou não uma doença.

 Para isso, vamos criar uma lista de valores binários, `y`, que será o equivalente a ter os _targets_ ("valores reais") de uma classificação binária.

De seguida vamos criar duas listas com alguns valores "invertidos", `y_hat_1` e `y_hat_2`, que será o equivalente a ter as previsões de dois classificadores diferentes para esses mesmos _targets_. Isto é, `y_hat_1` e `y_hat_2` são as previsões de dois classificadores diferentes para os mesmos _targets_ `y` onde alguns valores estão "errados".

### _Target_: y

Criemos então a lista de valores binários, `y`, que será o equivalente a ter os _targets_ ("valores reais") de uma classificação binária.

In [ ]:
# Gerar uma listas de 1000 números inteiros aleatórios entre 0 e 1
from scipy.stats import skewnorm

# número de observações (comprimento da lista com o "target")
n = 10000

# gerar uma lista de números aleatórios entre 0 e 1 em que uma das classes tem mais peso
peso_da_classe_0 = random.randint(10,90)
print(f"O peso da classe 0 é {peso_da_classe_0}%")
y = np.array(random.choices([0, 1], weights=[peso_da_classe_0, 100-peso_da_classe_0], k=n))

# plot histogram
import matplotlib.pyplot as plt
_ = plt.hist(y, bins=2)

Por exemplo, os primeiros 10 valores de y são:

In [ ]:
y[:10]

###  (Simulação de) previsões: y_hat_1 e y_hat_2
Na secção anterior definimos os valores de y, que são os _targets_ ("valores reais") de uma classificação binária.
P.e.:
- 0 significa que um email não é _spam_ e 1 significa que um email é _spam_
- 0 significa que um tumor é benigno e 1 significa que um tumor é maligno
- 0 significa que um cliente não vai comprar um produto e 1 significa que um cliente vai comprar um produto
- etc.

Definimos agora duas listas de valores, `y_hat_1` e `y_hat_2`, que será o equivalente a ter os _targets_ (`y`, "valores observados") e as previsões de dois classificadores diferentes para esses mesmos _targets_. Para isso, vamos usar uma função que inverte aleatoriamente "seed"% dos valores de y, em que invertido significa que 0 passa a 1 e 1 passa a 0.

## Definição da probabilidade de inversão dos valores de y (a menos de ruído)

Comecemos por definir a probabilidades de inversão dos valores de y (a menos de ruído).

In [ ]:
prob = string_para_int_entre_0_e_b(nomes_dos_alunos, 10)
f"A probabilidade de inversão dos valores de y é {prob}% (a menos de ruído que será adicionado mais à frente)."

### Calcula a inversão dos valores de y
A função `flip` recebe uma lista de valores binários (y) e inverte alguns desses valores, numa percentagem equivalente à probabilidade calculada anteriormente mais um ruído, i.e., a probabilidade de inversão (0->1 ou 1->0) é dada por:
$$ \text{probabilidade de inversão} = \text{prob} + \text{ruído} $$

In [ ]:
def flip(y, prob, ruido_max=10):
    """
    Função que inverte aleatoriamente "prob+rand(0,ruido)"% dos valores de y,
    em que invertido significa que 0 passa a 1 e 1 passa a 0.
    :param y: lista de valores
    :param prob: probabilidade de inversão
    :param ruido_max: ruído máximo a adicionar à probabilidade de inversão
    :return: lista com os valores de y invertidos
    """
    # criar uma cópia de y para não alterar o original
    y = y.copy()

    # o ruido é um número aleatório entre 0 e ruido_max que é adicionado a prob
    ruido = np.random.randint(0, ruido_max)

    # gerar uma lista de índices aleatórios
    mask = np.random.randint(0, n, (prob+ruido) * n//100)

    # inverter os valores de y nos índices gerados
    y[mask] = 1 - y[mask]

    return y

Usando a função anterior, vamos criar duas listas de valores invertidos, `y_hat_1` e `y_hat_2`.

In [ ]:
y_hat_1 = flip(y, prob=prob)

# os primeiros 10 valores de y_hat_1
y_hat_1[:10]

In [ ]:
y_hat_2 = flip(y, prob=prob)

# os primeiros 10 valores de y_hat_2
y_hat_2[:10]

Temos então os dados, `y`, `y_hat_1` e `y_hat_2`, que será o equivalente a ter os _targets_ ("valores observados/reais") e as previsões de dois classificadores diferentes.

Vamos colocar esses dados numa _DataFrame_ do `Pandas` para facilitar uma primeira análise.

In [ ]:
# concat y, y_hat_1 and y_hat_2 in a DataFrame
import pandas as pd
df = pd.DataFrame({
    'y': y,
    'y_hat_1': y_hat_1,
    'y_hat_1_correto': y == y_hat_1,
    'y_hat_2': y_hat_2,
    'y_hat_2_correto': y == y_hat_2,
})
df

Vamos na próxima secção analisar os resultados desses "classificadores" usando a matriz de confusão e as métricas de avaliação estudadas nas aulas teóricas.

# Questão 1: Matriz de confusão

Recorde que a matriz de confusão é dada por:
$$ \boxed{
\begin{matrix}
& \text{predição negativa} & \text{predição positiva} \\
\text{classe negativa} & TN & FP \\
\text{classe positiva} & FN & TP
\end{matrix}}
$$
em que
- TP: verdadeiros positivos
- FP: falsos positivos
- FN: falsos negativos
- TN: verdadeiros negativos
- TP + FP: número de predições positivas
- FN + TN: número de predições negativas
- TP + FN: número de valores reais positivos
- FP + TN: número de valores reais negativos
- TP + FP + FN + TN: número total de predições

Neste exercício pode usar a função [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) do sklearn.

Obs.: Em alguns livros e artigos, ou mesmo nos slides das aulas, a matriz de confusão é dada por:
$$ \boxed{
    \begin{matrix}
                                & \text{predição positiva}  & \text{predição negativa} \\
        \text{classe positiva}  & TP                        & FN \\
        \text{classe negativa}  & FP                        & TN
    \end{matrix}
    }
\text{ ou }
\boxed{
    \begin{matrix}
                                    & \text{classe positiva}    & \text{classe negativa} \\
        \text{predição positiva}    & TP                        & FP \\
        \text{predição negativa}    & FN                        & TN
    \end{matrix}
   }
$$

pelo que deve ter atenção à ordem das linhas e colunas da matriz de confusão.

## Questão 1.1: Calcule a matriz de confusão entre y e y_hat_1


In [ ]:
# TODO: matriz de confusão entre y e y_hat_1
matrix_confusao_1 = "TODO"

## Questão 1.2: Calcule a matriz de confusão entre y e y_hat_2

In [ ]:
# TODO: matriz de confusão entre y e y_hat_2
matrix_confusao_2 = "TODO"

## Questão 1.3: Que conclusões tirar a partir das matrizes de confusão?
Baseado nas matrizes de confusão, qual dos dois "classificadores" lhe parece melhor? Justifique.
Resposta: # TODO

# Questão 2: Acurácia, precisão, _recall_ e $F_1-score$
## Questão 2.1: Código próprio

Calcule a acurácia, precisão, _recall_ e $F_1-score$ do classificador. **Não use funções definidas em bibliotecas**, mas sim as fórmulas, i.e., calcule os valores a partir da matriz de confusão.

Lembre-se que:
$$ \text{acurácia} = \frac{TP + TN}{TP + TN + FP + FN} $$
$$ \text{precisão} = \frac{TP}{TP + FP} $$
$$ \text{recall} = \frac{TP}{TP + FN} $$
$$ F_1 = 2 \frac{\text{precisão} \times \text{recall}}{\text{precisão} + \text{recall}} $$

In [ ]:
def valores_da_matriz_de_confusao(mat):
    TP = mat[1, 1]
    TN = mat[0, 0]
    FP = mat[0, 1]
    FN = mat[1, 0]
    return TP, TN, FP, FN

TP_1, TN_1, FP_1, FN_1 = valores_da_matriz_de_confusao(matrix_confusao_1)
TP_2, TN_2, FP_2, FN_2 = valores_da_matriz_de_confusao(matrix_confusao_2)

Confirmacao que temos os valores da matriz de confusao original

In [ ]:
ok_1 = [[TN_1, FP_1], [FN_1, TP_1]] == matrix_confusao_1
ok_2 = [[TN_2, FP_2], [FN_2, TP_2]] == matrix_confusao_2

print(f"Os valores da matriz de confusão 1 estão corretos? \n{ok_1}")
print(f"Os valores da matriz de confusão 2 estão corretos? \n{ok_2}")

In [ ]:
# TODO: calcular acurácia
def accuracy(TP, TN, FP, FN):
    return "TODO"

In [ ]:
# TODO: calcular precisão
def precision(TP, FP):
    return "TODO"

In [ ]:
# TODO: calcular recall
#...

In [ ]:
# TODO: calcular F1-score
#...

In [ ]:
f"A acurácia para o classificador 1 foi: {accuracy(TP_1, TN_1, FP_1, FN_1):.3f} e para o classificador 2 foi: {accuracy(TP_2, TN_2, FP_2, FN_2):.3f}"

In [ ]:
f"A precisão para o classificador 1 foi: {precision(TP_1, FP_1):.3f} e para o classificador 2 foi: {precision(TP_2, FP_2):.3f}"

In [ ]:
f"O recall para o classificador 1 foi: {recall(TP_1, FN_1):.3f} e para o classificador 2 foi: {recall(TP_2, FN_2):.3f}"

In [ ]:
f"O F1-score para o classificador 1 foi: {f1_score(precision(TP_1, FP_1), recall(TP_1, FN_1)):.3f} e para o classificador 2 foi: {f1_score(precision(TP_2, FP_2), recall(TP_2, FN_2)):.3f}"

## Questão 2.2: Usando o sklearn
Valide os seus código com as funções do sklearn, i.e., compare os resultados das suas funções com os resultados das funções do sklearn

In [ ]:
# TODO: validar os resultados com as funções do sklearn

# TODO: importar as funções do sklearn para calcular as métricas de avaliação: accuracy_score, precision_score, recall_score, f1_score

# TODO: calcular as métricas de avaliação para o classificador 1

# TODO: calcular as métricas de avaliação para o classificador 2

# TODO: comparar os resultados

## Questão 2.3: Que conclusões tirar a partir dos valores de acurácia, precisão, _recall_ e $F_1-score$?
Baseado na acurácia, precisão, _recall_ e $F_1-score$, qual dos dois "classificadores" lhe parece melhor? Justifique.

Resposta: # TODO

## Questão 2.4: Que conclusões tirar a partir dos valores de acurácia, precisão, _recall_ e $F_1-score$ supondo que os casos positivos (1) correspondem a emails _spam_?
Resposta: # TODO

## Questão 2.5: Que conclusões tirar a partir dos valores de acurácia, precisão, _recall_ e $F_1-score$ supondo que os casos positivos (1) correspondem a casos de doença?
Resposta: # TODO